# Transformata Fouriera dla obrazów cyfrowych. Filtracja w dziedzinie częstotliwości.

## Cel ćwiczenia

- Zapoznanie z wykorzystaniem transformaty Fouriera w przetwarzaniu obrazów cyfrowych.
- Zapoznanie z pojęciem F-obrazu (amplitudy i fazy).
- Zapoznanie z własnościami transformaty Fouriera.
- Zapoznanie z filtracją w dziedzinie częstotliwości.

Na jednym z poprzednich ćwiczeń zetknęliśmy się z pojęciem konwolucji.
Przykładem może być filtracja dolno i górnoprzepustowa.
Operacja ta odpowiada mnożeniu w dziedzinie częstotliwości zgodnie z zależnością:

\begin{equation}
\mathcal{F}(g(x,y)*h(x,y)) = \mathcal{F}(g(x,y)) \cdot \mathcal{F}(h(x,y))
\end{equation}

gdzie: $\mathcal{F}$ oznacza transformatę Fouriera, a $*$ jest splotem.

Operacja filtracji w dziedzinie częstotliwości może okazać się bardziej efektywna, jeżeli operacje $fft$ i $ifft$ (odpowiednio szybka transformata Fouriera -- *fast Fourier transform* -- oraz odwrotna szybka transformata Fouriera -- *inverse fast Fourier transform*) zajmą mniej czasu niż klasyczna konwolucja (zazwyczaj ma to miejsce dla dużego obrazu, z dużą maską).

Sama filtracja w dziedzinie częstotliwości to mnożenie punktowe całego obrazu przez jedną maskę.

W przypadku filtracji w dziedzinie częstotliwości zakłada się, że obraz "zawija się" na brzegach, co może powodować pewne artefakty (zostanie to pokazane w trakcie ćwiczenia).

W dziedzinie częstotliwości "działają" tylko filtry liniowe.
Filtry medianowe, maksymalne, minimalne itp. nie mają swoich odpowiedników.

## Dwuwymiarowa transformata Fouriera

1. Wczytaj plik "dwieFale.bmp" w skali szarości.
Jest to obraz powstały na podstawie następującej zależności:

\begin{equation}
L(m, n) = 128 + 127 \cdot \cos(\frac{2\pi m}{32}+\frac{3\pi}{4}) \cdot \cos(\frac{2\pi n}{8}-\frac{\pi}{2})
\end{equation}<br>

gdzie: $m$ i $n$ są odpowiednio numerami wierszy i kolumn.

2. Do realizacji dwuwymiarowej transformaty Fouriera służy funkcja `cv2.dft`.
Ustaw flagę `flags=cv2.DFT_COMPLEX_OUTPUT`.
Wykonaj transformatę na wczytanym obrazie.
W ten sposób uzyskuje się tzw. F-obraz.

3. Najniższe częstotliwości znajdują się w lewym-górnym rogu obrazu.
Dla celów wizualizacji (ale też przetwarzania) często wykonuje się tzw. przesunięcie F-obrazu, które powoduje, że niskie częstotliwości przesuwane są do środka obrazu.
Wykorzystaj funkcję `np.fft.fftshift`.
Jako pierwszy argument podaj wynik transformaty Fouriera.
Jako drugi argument podaj numery osi, wzdłuż których należy wykonać operację.
Pierwsza oś odnosi się do wierszy obrazu.
Druga oś odnosi się do kolumn obrazu.
Trzecia oś to część rzeczywista (`[:, :, 0]`) lub urojona (`[:, :, 1]`).
W naszym przypadku argument powinien wyglądać tak `[0,1]`.

4. Wyświetl wynik transformaty.
Na wspólnym wykresie umieść obraz oryginalny, amplitudę i fazę F-obrazu.
Amplitudę i fazę wyznacz za pomocą funkcji `cv2.cartToPolar`.
Pierwszym argumentem funkcji jest część rzeczywista wyniku, a drugim urojona.
Uwaga. W razie wątpliwości proszę sprawdzić rozmiary rezultatu transformaty Fouriera oraz przesunięcia.

5. Dla wizualizacji oblicz logarytm dziesiętny amplitudy: `ALog = np.log10(A + 1)`.
Wyświetl go zamiast amplitudy na poprzednim wykresie.

6. Wczytaj obrazy *kolo.bmp*, *kwadrat.bmp*, *kwadrat45.bmp*, *trojkat.bmp*.
Czy analizując F-obraz można coś powiedzieć o kierunku krawędzi obiektów?

7. Sprawdź (empirycznie) poprawność stwierdzenia:
>Dwuwymiarowa transformata Fouriera jest złożeniem dwóch transformat jednowymiarowych (wykonanych np. najpierw wierszowo, a później kolumnowo). Jednowymiarowa transformata realizowana jest za pomocą funkcji fft (z bibloteki Numpy).
>
Wykonaj najpierw transformatę po wierszach: `FRow = np.fft.fft(I, axis=0)`.
Następnie po kolumnach: `FCol = np.fft.fft(FRow, axis=1)`.
Numpy zwraca wynik jako tablicę liczb zespolonych.
Część rzeczywistą można otrzymać w następujący sposób: `FCol.real`, a urojoną: `FCol.imag`.
Porównaj tak uzyskany wynik z rezultatem działania funkcji `cv2.dft`.
Można to zrobić wizualnie lub z wykorzystaniem funkcji `cv2.absdiff`.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

# Load required files
if not os.path.exists("dwieFale.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/dwieFale.bmp --no-check-certificate
if not os.path.exists("kolo.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/kolo.bmp --no-check-certificate
if not os.path.exists("kwadrat.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/kwadrat.bmp --no-check-certificate
if not os.path.exists("kwadrat45.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/kwadrat45.bmp --no-check-certificate
if not os.path.exists("kwadratKL.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/kwadratKL.bmp --no-check-certificate
if not os.path.exists("kwadratS.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/kwadratS.bmp --no-check-certificate
if not os.path.exists("kwadratT.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/kwadratT.bmp --no-check-certificate
if not os.path.exists("lena.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/lena.bmp --no-check-certificate
if not os.path.exists("trojkat.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/trojkat.bmp --no-check-certificate
if not os.path.exists("literki.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/literki.bmp --no-check-certificate
if not os.path.exists("wzorA.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/wzorA.bmp --no-check-certificate

I_Fale = cv2.imread('dwieFale.bmp', cv2.IMREAD_GRAYSCALE)

figFale, axsFale = plt.subplots()

axsFale.imshow(I_Fale, 'gray', vmin=0, vmax=256)
axsFale.axis('off')

figFale.show()

In [ ]:
# funkcja do plotowania 3D
def plot_mat(mat: np.ndarray, title: str='Matrix as surface', color: str='gray', projection: tuple=(35,30)):
    
    # get shape
    n,m = mat.shape

    # create mesh
    X, Y = np.meshgrid(np.arange(0,n,1), np.arange(0,m,1))

    # plot the stuff
    fig, ax = plt.subplots(subplot_kw={'projection':'3d'}, figsize=(10,10))
    
    if color == 'gray':
        surf = ax.plot_wireframe(X,Y,mat, color=color, rstride=1, cstride=1)
    else:
        surf = ax.plot_surface(X,Y,mat, cmap=color, rstride=1, cstride=1, edgecolor='none')
    ax.set_title(title)
    ax.view_init(*projection)


In [ ]:
def plot_Fimg(img: np.ndarray, title: str='polar decomp of dft of input', cmap: str='gray', plot3d: bool=True):
    fimg = cv2.dft(np.float32(img), flags=cv2.DFT_COMPLEX_OUTPUT)
    fimg = np.fft.fftshift(fimg, [0,1])
    mag, phase = cv2.cartToPolar(fimg[:,:,0], fimg[:,:,1])
    if plot3d:
        plot_mat(fimg[:,:,0], title='Real part', color=cmap, projection=(20, 15))
        plot_mat(fimg[:,:,1], title='Imaginary part', color=cmap, projection=(20, -15))
    fig, ax = plt.subplots(1,3, figsize=(15, 5))
    ax[0].imshow(img, 'gray'), ax[0].axis('off'), ax[0].set_title('Original image')
    ax[1].imshow(mag, 'gray'), ax[1].set_title('Magnitude'), ax[1].axis('off')
    ax[2].imshow(phase, 'gray'), ax[2].set_title('Phase'), ax[2].axis('off')
    fig.suptitle(title)

def plot_Fimg_logmag(img: np.ndarray, title: str='polar decomp of dft of input with log of magnitude'):
    fimg = cv2.dft(np.float32(img), flags=cv2.DFT_COMPLEX_OUTPUT)
    fimg = np.fft.fftshift(fimg, [0,1])
    mag, phase = cv2.cartToPolar(fimg[:,:,0], fimg[:,:,1])
    magLog = np.log10(mag+1)
    fig, ax = plt.subplots(1,3, figsize=(15, 5))
    ax[0].imshow(img, 'gray'), ax[0].axis('off'), ax[0].set_title('Original image')
    ax[1].imshow(magLog, 'gray'), ax[1].set_title('Logarithm of magnitude'), ax[1].axis('off')
    ax[2].imshow(phase, 'gray'), ax[2].set_title('Phase'), ax[2].axis('off')
    fig.suptitle(title)

In [ ]:
plot_mat(I_Fale, 'Fale image wireframe plot')

In [ ]:
# 2. Do realizacji dwuwymiarowej transformaty Fouriera służy funkcja cv2.dft. Ustaw flagę flags=cv2.DFT_COMPLEX_OUTPUT. Wykonaj transformatę na wczytanym obrazie. W ten sposób uzyskuje się tzw. F-obraz.
# 3. Najniższe częstotliwości znajdują się w lewym-górnym rogu obrazu. Dla celów wizualizacji (ale też przetwarzania) często wykonuje się tzw. przesunięcie F-obrazu, które powoduje, że niskie częstotliwości przesuwane są do środka obrazu. Wykorzystaj funkcję np.fft.fftshift. Jako pierwszy argument podaj wynik transformaty Fouriera. Jako drugi argument podaj numery osi, wzdłuż których należy wykonać operację. Pierwsza oś odnosi się do wierszy obrazu. Druga oś odnosi się do kolumn obrazu. Trzecia oś to część rzeczywista ([:, :, 0]) lub urojona ([:, :, 1]). W naszym przypadku argument powinien wyglądać tak [0,1].
# 4. Wyświetl wynik transformaty. Na wspólnym wykresie umieść obraz oryginalny, amplitudę i fazę F-obrazu. Amplitudę i fazę wyznacz za pomocą funkcji cv2.cartToPolar. Pierwszym argumentem funkcji jest część rzeczywista wyniku, a drugim urojona. Uwaga. W razie wątpliwości proszę sprawdzić rozmiary rezultatu transformaty Fouriera oraz przesunięcia.
# 5. Dla wizualizacji oblicz logarytm dziesiętny amplitudy: ALog = np.log10(A + 1). Wyświetl go zamiast amplitudy na poprzednim wykresie.
plot_Fimg(I_Fale)
plot_Fimg_logmag(I_Fale)

In [ ]:
# 6. Wczytaj obrazy kolo.bmp, kwadrat.bmp, kwadrat45.bmp, trojkat.bmp. Czy analizując F-obraz można coś powiedzieć o kierunku krawędzi obiektów?
kolo = cv2.imread('kolo.bmp', cv2.IMREAD_GRAYSCALE)
kwadrat = cv2.imread('kwadrat.bmp', cv2.IMREAD_GRAYSCALE)
kwadrat45 = cv2.imread('kwadrat45.bmp', cv2.IMREAD_GRAYSCALE)
trojkat = cv2.imread('trojkat.bmp', cv2.IMREAD_GRAYSCALE)

In [ ]:
# obraz kolo
plot_Fimg(kolo, cmap='viridis')
plot_Fimg_logmag(kolo)

In [ ]:
# obraz kwadrat
plot_Fimg(kwadrat, cmap='viridis')
plot_Fimg_logmag(kwadrat)

In [ ]:
# obraz kwadrat45
plot_Fimg(kwadrat45, cmap='viridis')
plot_Fimg_logmag(kwadrat45)

In [ ]:
# obraz trojkat
plot_Fimg(trojkat, cmap='viridis')
plot_Fimg_logmag(trojkat)

In [ ]:
# 7. Sprawdź (empirycznie) poprawność stwierdzenia:
#Dwuwymiarowa transformata Fouriera jest złożeniem dwóch transformat jednowymiarowych (wykonanych np. najpierw wierszowo, a później kolumnowo). Jednowymiarowa transformata realizowana jest za #pomocą funkcji fft (z bibloteki Numpy).
#Wykonaj najpierw transformatę po wierszach: FRow = np.fft.fft(I, axis=0). Następnie po kolumnach: FCol = np.fft.fft(FRow, axis=1). Numpy zwraca wynik jako tablicę liczb zespolonych. Część #rzeczywistą można otrzymać w następujący sposób: FCol.real, a urojoną: FCol.imag. Porównaj tak uzyskany wynik z rezultatem działania funkcji cv2.dft. Można to zrobić wizualnie lub z #wykorzystaniem funkcji cv2.absdiff.
img = kolo

# dft from cv2 module
dft_kolo = cv2.dft(np.float32(img), flags=cv2.DFT_COMPLEX_OUTPUT)
dft_kolo_rel = dft_kolo[:,:,0]
dft_kolo_img = dft_kolo[:,:,1]

# fft from numpy module
fft_row = np.fft.fft(img, axis=0)
fft_kolo = np.fft.fft(fft_row, axis=1)

# check if the real and imaginary parts are equal
mse_rel = (np.square(dft_kolo_rel - fft_kolo.real)).mean(axis=None)
mse_img = (np.square(dft_kolo_img- fft_kolo.imag)).mean(axis=None)

# print the error
print('Real error     : {:.8f}\nImaginary error: {:.8f}'.format(mse_rel, mse_img))

# show the absolute difference between the images
fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].imshow(np.abs(dft_kolo_rel - fft_kolo.real), 'gray'), ax[0].axis('off'), ax[0].set_title('Real part difference')
_ = ax[1].imshow(np.abs(dft_kolo_img - fft_kolo.imag), 'gray'), ax[1].axis('off'), ax[1].set_title('Imaginary part difference')

## Wnioski i obserwacje:
- działanie transformaty Fouriera najepierw na rzędach, a potem na kolumnach daje (prawie) taki sam efekt jak działanie dwuwymiarowej transformaty.

## Własności dwuwymiarowej transformaty Fouriera

1. Zbadaj jak zmienia się F-obraz (amplituda i faza) podczas następujących operacji: translacja, rotacja, zmiana rozmiaru, kombinacja liniowa.
Wykorzystaj stworzony wcześniej kod.<br>
Uwaga. Należy użyć przygotowanych obrazów, a nie "generować" własne.
2. Do badania translacji wykorzystaj obrazy *kwadrat.bmp* i *kwadratT.bmp*.
3. Przy badaniu rotacji wykorzystaj obrazy *kwadrat.bmp* i *kwadrat45.bmp*.
4. Przy badaniu zmiany rozmiaru wykorzystaj obrazy *kwadrat.bmp* i *kwadratS.bmp*.
5. Przy badaniu kombinacji liniowej wykorzystaj obrazy *kwadrat.bmp*, *kwadrat45.bmp* i *kwadratKL.bmp*.

In [ ]:
# oryginalny kwadrat
plot_Fimg(kwadrat, plot3d=False)

In [ ]:
# translacja
kwadratT = cv2.imread('kwadratT.bmp', cv2.IMREAD_GRAYSCALE)
plot_Fimg(kwadratT, plot3d=False)

In [ ]:
# rotacja
plot_Fimg(kwadrat45, plot3d=False)

In [ ]:
# skalowanie
kwadratS = cv2.imread('kwadratS.bmp', cv2.IMREAD_GRAYSCALE)
plot_Fimg(kwadratS, plot3d=False)

In [ ]:
# kombinacja liniowa
kwadratKL = cv2.imread('kwadratKL.bmp', cv2.IMREAD_GRAYSCALE)
plot_Fimg(kwadratKL, plot3d=False)

## Wnioski i obserwacje:
- translacja powoduje zmianę fazy w coś przypominającego poprzeczne paski w każdej ćwiartce -> przesunięcie o jakiś kąt,
- rotacja powoduje w fazie niejako przesunięcie i utworzenie dwóch 'przekątnych kwadratu' -> obrót o jakiś kąt,
- skalowanie organizuje fazę w jednorodny wzór po przekątnej -> zmniejsza wsp skalowania kąta przy sinusach/kosinusach,
- kombinacja liniowa transformacji daje efekt składowych kombinacji w wykresie fazy.

## Odwrotna dwuwymiarowa transformata Fouriera

1. Wykorzystaj stworzony wcześniej kod. Wybierz dowolny obraz np "kolo.bmp".
2. Przed realizacją odwrotnego przekszałcenia należy wykonać odwrotne przesunięcie.
Wykorzystaj funkcję `np.fft.ifftshift`.
Pierwszym argumentem jest wynik transformaty Fouriera.
Drugim argumentem są numery osi, wzdłuż których należy wykonać operację.
3. Wykonaj odwrotną transformatę Fouriera za pomocą funkcji `cv2.idft`.
Jako drugi argument przekaż następujące flagi: `flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT`.
Wynik może mieć małą część urojoną przez błędy numeryczne.
Aby pozbyć się tego efekty należy obliczyć amplitudę:
        imgIFFT = cv2.magnitude(ifft[:, :, 0], ifft[:, :, 1])
Następnie wynik należy zaokrąglić (`np.round`) i zrzutować do typu `uint8`.
4. Wyświetl wynik.
Sprawdź (wizualnie i poprzez odjęcie) czy obraz oryginalny i po przekształceniach są takie same.

In [ ]:
# calculate the transform
kolo_fft = cv2.dft(np.float32(kolo), flags=cv2.DFT_COMPLEX_OUTPUT)

# shift the transform
kolo_shift = np.fft.fftshift(kolo_fft, [0,1])

# display the shifted dft
fig, ax = plt.subplots(1, 2, figsize=(10,5))
ax[0].imshow(kolo_shift[:,:,0], 'gray'), ax[0].axis('off'), ax[0].set_title('dft of kolo (real part)')
_ = ax[1].imshow(kolo_shift[:,:,1], 'gray'), ax[1].axis('off'), ax[1].set_title('dft of kolo (imaginary part)')

# shift the image back
kolo_backshift = np.fft.ifftshift(kolo_fft, [0,1])

# calculate the inverse dft
kolo_ifft = cv2.idft(kolo_backshift, flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
kolo_ifft = np.round(cv2.magnitude(kolo_ifft[:,:,0], kolo_ifft[:,:,1]))
kolo_ifft = np.array(kolo_ifft, dtype=np.uint8)

# display the iverse transform and compare to the original image
fig, ax = plt.subplots(1, 3, figsize=(15,5))
ax[0].imshow(kolo, 'gray'), ax[0].axis('off'), ax[0].set_title('Original')
ax[1].imshow(kolo_ifft, 'gray'), ax[1].axis('off'), ax[1].set_title('After dft and inverse dft')
_ = ax[2].imshow(np.abs(kolo_ifft-kolo), 'gray'), ax[2].axis('off'), ax[2].set_title('The difference')

# calculate the mean square error and print it
# check if the real and imaginary parts are equal
mse_ifft = (np.square(kolo - kolo_ifft)).mean(axis=None)

# print the error
print('Mean square error between both images: {:.17f}'.format(mse_ifft))

## Wnioski, obserwacje:
- błąd średniokwadratowy jest na poziomie zera maszynowego (mniej niż $1.0\cdot E^{-17}$),
- nie ma widocznych różnic między obrazami po tych wszystkich przekształceniach.

## Filtracja obrazu w dziedzinie częstotliwości

1. Wczytaj obraz "lena.bmp" w skali szarości.
Wykonaj transformatę Fouriera.
Wykorzystaj stworzony poprzednio kod.
Wyświetl obraz oryginalny, amplitudę (w skali logarytmicznej) i fazę.

2. Przeprowadź filtrację dolnoprzepustową - usuń górne częstotliwości.
Dla F-obrazu po operacji przesunięcia (`fftshift`) niskie częstotliwości leżą w jego centrum.

3. Na początku stwórz filtr "kołowy", dolnoprzepustowy.
Należy wygenerować macierze opisujące przestrzeń w dziedzinie częstotliwości.
Ich rozmiar musi być taki sam jak rozmiar przetwarzanego obrazu.
        lenaSize = I_Lena.shape
        FSpaceRows = 2 * np.fft.fftshift(np.fft.fftfreq(lenaSize[0]))
        FSpaceRowsM = np.outer(FSpaceRows, np.ones([1, lenaSize[1]]))
        FSpaceCols = 2 * np.fft.fftshift(np.fft.fftfreq(lenaSize[1]))
        FSpaceColsM = np.outer(np.ones([1, lenaSize[0]]), FSpaceCols)
Powyższy kod wygeneruje dwie znormalizowane macierze częstotliwości: *FSpaceRowsM* i *FSpaceColsM*.
Następnie należy wyznaczyć macierz zawierającą "odległość" od składowej stałej.
        FreqR = np.sqrt(np.square(FSpaceRowsM) + np.square(FSpaceColsM))

Uwagi:
- funkcja `fftfreq` generuje wektor częstotliwości $[-0.5, 0.5]$ o określonym rozmiarze, przy czym układ wartości jest taki, że najpierw od 0 do 0.5, a potem od -0.5 do 0,
- operacja `fftshift` zmienia ten układ na $[-0.5, 0.5]$,
- mnożenie przez 2 ustala ostatecznie zakres na $[-1, 1]$,
- operacja `outer` to tzw. iloczyn zewnętrzy dwóch wektorów, w naszym przypadku powoduje, że wektor pionowy lub poziomy jest "powielany" odpowiednią liczbę razy.   
- sugeruje się, aby przyglądnąć się jak wygląda macierz `FreqR` - czy to w debugerze, czy poprzez wizualizację.

4. Teraz należy wybrać interesujący zakres.
Tu można zdefiniować typ filtru (dolno, górno, pasmowoprzepustowy).

        FilterF = FreqR <= 0.1

Filtr należy zwizualizować:

        figFilter = plt.figure()
        axsFilter = figFilter.add_subplot(projection='3d')
        axsFilter.plot_surface(FSpaceRowsM, FSpaceColsM, FilterF, rstride=1, cstride=1, cmap=plt.get_cmap('gray'), linewidth=0)
        figFilter.show()

4. Wykonaj właściwą filtrację, czyli mnożenie F-obrazu przez filtr FilterF.
Trzeba pamiętać, że F-obraz ma 2 kanały (rzeczywisty i urojony).
By mnożenie było możliwe należy więc powielić filtr również na 2 kanały.

        FilterF3 = np.repeat(FilterF[:, :, np.newaxis], 2, axis=2)

5. Wykonaj operację odwrotnego przesunięcia i odwrotnej transformaty.
Oblicz wartość bezwzględną wyniku.
Wykorzystaj funkcję `cv2.magnitude`.
Pierwszym argumentem jest część rzeczywista.
Drugim argumentem jest część urojona.
Wynik wyświetl.

6. Poeksperymentuj z rozmiarem filtru (promieniem).
Zaimplementuj filtr górnoprzepustowy (zmiana znaku przy warunku na odległość) oraz pasmowoprzepustowy (dwa warunki na promień połączone operatorem AND '&' ).
Wykonaj co najmniej trzy filtry i wyświetl wyniki.

7. W ten sposób zaimplementowana filtracja wprowadza pewne artefakty w postaci "pierścieni" wokół krawędzi.
Zapobiec temu zjawisku można poprzez odpowiednie "modelowanie" filtra.
W tym celu wykorzystać należy okna, np. Hamminga, Hanninga, Chebysheva (znane z przetwarzania sygnałów 1D).
Zagadnienie to jest tematem zadania domowego do tego ćwiczenia.

In [ ]:
def low_filter(src, filter_type='lowpass', r=0.1):
    srcDFT = cv2.dft(np.float32(src),flags=cv2.DFT_COMPLEX_OUTPUT)
    srcDFT = np.fft.fftshift(srcDFT, [0,1])

    size = src.shape
    FSpaceRows = 2 * np.fft.fftshift(np.fft.fftfreq(size[0]))
    FSpaceRowsM = np.outer(FSpaceRows, np.ones([1, size[1]]))
    FSpaceCols = 2 * np.fft.fftshift(np.fft.fftfreq(size[1]))
    FSpaceColsM = np.outer(np.ones([1, size[0]]), FSpaceCols)
    
    FreqR = np.sqrt(np.square(FSpaceRowsM) + np.square(FSpaceColsM))
    
    FilterF = FreqR <= r

    figFilter = plt.figure()
    axsFilter = figFilter.add_subplot(projection='3d')
    axsFilter.plot_surface(FSpaceRowsM, FSpaceColsM, FilterF, rstride=1, cstride=1, cmap=plt.get_cmap('gray'), linewidth=0)
    
    FilterF3 = np.repeat(FilterF[:, :, np.newaxis], 2, axis=2)
    
    srcDFT = srcDFT*FilterF3
    srcIFFT = np.fft.ifftshift(srcDFT, [0,1])
    dst = cv2.idft(srcIFFT, flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
    dst = cv2.magnitude(dst[:, :, 0], dst[:, :, 1])
    dst = np.round(dst).astype('uint8')

    return dst

def high_filter(src, r=0.7):
    srcDFT = cv2.dft(np.float32(src),flags=cv2.DFT_COMPLEX_OUTPUT)
    srcDFT = np.fft.fftshift(srcDFT, [0,1])

    size = src.shape
    FSpaceRows = 2 * np.fft.fftshift(np.fft.fftfreq(size[0]))
    FSpaceRowsM = np.outer(FSpaceRows, np.ones([1, size[1]]))
    FSpaceCols = 2 * np.fft.fftshift(np.fft.fftfreq(size[1]))
    FSpaceColsM = np.outer(np.ones([1, size[0]]), FSpaceCols)
    
    FreqR = np.sqrt(np.square(FSpaceRowsM) + np.square(FSpaceColsM))
    
    
    FilterF = FreqR >= r
    
    figFilter = plt.figure()
    axsFilter = figFilter.add_subplot(projection='3d')
    axsFilter.plot_surface(FSpaceRowsM, FSpaceColsM, FilterF, rstride=1, cstride=1, cmap=plt.get_cmap('gray'), linewidth=0)
    
    FilterF3 = np.repeat(FilterF[:, :, np.newaxis], 2, axis=2)
    
    srcDFT = srcDFT*FilterF3
    srcIFFT = np.fft.ifftshift(srcDFT, [0,1])
    dst = cv2.idft(srcIFFT, flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
    dst = cv2.magnitude(dst[:, :, 0], dst[:, :, 1])
    dst = np.round(dst).astype('uint8')

    return dst

def bandpass_filter(src, r_low=0.1, r_high=0.7):
    
    dst = low_filter(src, r_low)
    dst = high_filter(src, r_high)

    return dst

In [ ]:
# load lena
lena = cv2.imread('lena.bmp', cv2.IMREAD_GRAYSCALE)

In [ ]:
# filter the image
lena_lowpass = low_filter(lena)

# display the image and the differences
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(lena, 'gray'), ax[0].axis('off'), ax[0].set_title('Original')
ax[1].imshow(lena_lowpass, 'gray'), ax[1].axis('off'), ax[1].set_title('After low pass filtration')
_ = ax[2].imshow(np.abs(lena-lena_lowpass), 'gray'), ax[2].axis('off'), ax[2].set_title('The difference')

In [ ]:
# filter the image
lena_bandpass = bandpass_filter(lena)

# display the image and the differences
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(lena, 'gray'), ax[0].axis('off'), ax[0].set_title('Original')
ax[1].imshow(lena_bandpass, 'gray'), ax[1].axis('off'), ax[1].set_title('After band pass filtration')
_ = ax[2].imshow(np.abs(lena-lena_bandpass), 'gray'), ax[2].axis('off'), ax[2].set_title('The difference')

In [ ]:
# filter the image
lena_highpass = high_filter(lena)

# display the image and the differences
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(lena, 'gray'), ax[0].axis('off'), ax[0].set_title('Original')
ax[1].imshow(lena_highpass, 'gray'), ax[1].axis('off'), ax[1].set_title('After high pass filtration')
_ = ax[2].imshow(np.abs(lena-lena_highpass), 'gray'), ax[2].axis('off'), ax[2].set_title('The difference')`

## Implementacja wyszukiwania wzorca za pomocą FFT

1. Wczytaj w skali szarości i wyświetl obrazy *literki.bmp* i *wzorA.bmp*.

2. Wyznacz transformatę Fouriera obrazu *literki.bmp*.

3. Obróć drugi obraz o $180^\circ$.
Zastosuj funkcję `np.rot90`.
Pierwszym argumentem jest obracana macierz, a drugim liczba obrotów o $90^\circ$.

4. Należy wyznaczyć transformatę Fouriera obróconego obrazu w taki sposób, żeby miała ona taki sam rozmiar jak pierwszy obraz.
W tym celu należy zastosować *Zero Padding*.
Operacja ta polega na uzupełnieniu obrazu zerami do oczekiwanego rozmiaru.
Uzupełnij obraz zerami z **prawej** strony i z **dołu**.
W tym celu należy wykorzystać funkcję `cv2.copyMakeBorder`.
    - Pierwszym argumentem jest obraz wejściowy.
    - Drugim argumentem jest liczba wierszy u góry.
    - Trzecim argumentem jest liczba wierszy u dołu.
    - Czwartym argumentem jest liczba kolumn z lewej.
    - Piątym argumentem jest liczba kolumn z prawej.
    - Szóstym argumentem jest flaga typu wypełnienia.
    Dla stałej wartości podaj `cv2.BORDER_CONSTANT`.
    - Siódmym argementem jest wartość pikseli w ramce.
    Przekaż `value=0`.

5. Wyznacz transformatę Fouriera obrazu stworzonego w poprzednim punkcie.

6. Wyniki obu transformat należy przekonwertować do liczb zespolonych.
Obecnie jest to dwukanałowa macierz.
Pierwszy kanał odpowiada za część rzeczywistą.
Drugi kanał odpowiada za część urojoną.
Aby to osiągnąć wystarczy wykonać działanie:
        Complex = Real + Imag * 1j

7. Przemnóż ze sobą zespolone wyniki transformat.

8. Wynik należy powrotnie przekształcić do dwukanałowej macierzy.
Aby to zrobić wykonaj operację:
        CompMat = cv2.merge([np.real(Complex), np.imag(Complex)])

9. Wykonaj odwrotną transformatę Fouriera.
Dodaj flagę `flags=cv2.DFT_COMPLEX_INPUT`.

10. Oblicz wartość bezwzględną wyniku.

11. Wykonaj morfologiczną operację **Top-Hat**, by znaleźć maksima lokalne.
Operacja ta zostanie dokładnej wyjaśniona w jednym z kolejnych ćwiczeń.
W tym celu wykorzystaj operację:
        cv2.morphologyEx(correlation, cv2.MORPH_TOPHAT, np.ones((3, 3), np.uint8))

12. Wyświetl obok siebie obraz wejściowy i wynik wykonanych operacji.
Czy możesz wskazać położenie wzoru na podstawie drugiego obrazu?

In [ ]:
liter = cv2.imread('literki.bmp', cv2.IMREAD_GRAYSCALE)
wzora = cv2.imread('wzorA.bmp', cv2.IMREAD_GRAYSCALE)

In [ ]:
dft_liter = cv2.dft(np.float32(liter), flags=cv2.DFT_COMPLEX_OUTPUT)
dft_liter = np.fft.fftshift(dft_liter, [0,1])

In [ ]:
wzora_pi_flip = np.rot90(wzora, 2)

In [ ]:
n0, m0 = wzora_pi_flip.shape
n1, m1 = liter.shape
wzora_pi_flip_pad = cv2.copyMakeBorder(wzora_pi_flip, 0, n1-n0, 0, m1-m0, cv2.BORDER_CONSTANT, value=0)
pad = wzora_pi_flip_pad

In [ ]:
dft_pad = cv2.dft(np.float32(pad), flags=cv2.DFT_COMPLEX_OUTPUT)
dft_pad = np.fft.fftshift(dft_pad, [0,1])

In [ ]:
real_pad = dft_pad[:,:,0]
imag_pad = dft_pad[:,:,1]
Complex_pad = real_pad + imag_pad*1j

In [ ]:
real_liter = dft_liter[:,:,0]
imag_liter = dft_liter[:,:,1]
Complex_liter = real_liter + imag_liter*1j

In [ ]:
Complex = Complex_pad*Complex_liter

In [ ]:
CompMat = cv2.merge((np.real(Complex), np.imag(Complex)))

In [ ]:
correlation = cv2.idft(CompMat, flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT | cv2.DFT_COMPLEX_INPUT)
correlation = np.fft.ifftshift(correlation, [0, 1])
correlation = np.abs(correlation)

In [ ]:
search = cv2.morphologyEx(correlation, cv2.MORPH_TOPHAT, np.ones((3, 3), np.uint8))
search = cv2.magnitude(search[:,:,0], search[:,:,1])

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,5.2))
ax[0].imshow(liter, 'gray'), ax[0].axis('off')
ax[1].imshow(search, 'gray'), ax[1].axis('off')
_ = fig.suptitle('Template detection in an image')

## Wnioski i obserwacje:
- udało się dokonać rozpoznania wzoru, jednak wynik ten bynajmniej nie jest znakomity,
- metoda ta zaznaczyła na obrazie wiele krawędzi, które pokrywają się z tymi ze wzoru.